In [ ]:
import numpy as np
import cv2
import dlib

class ImageFeatureExtractor:
    def __init__(self, dlibShapePredictorFilepath, dlibPoints):
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(dlibShapePredictorFilepath)
        self.dlibPoints = dlibPoints
    
    def extractFeatures(self, image):

        # detect the face bounding box
        rects = self.detector(image, 1)

        # extract the first face
        if len(rects) == 0:
            raise Exception("Could not find any faces in the image!!")
        rect = rects[0]

        # arrays for the x and y feature components
        features_x = []
        features_y = []

        # extract each faeture point
        shape = self.predictor(image, rect)
        for i in range(shape.num_parts):
            features_x.append(shape.part(i).x)
            features_y.append(shape.part(i).y)

        # extract only the requested dlib points, indexed at 1
        features_x = np.array(features_x)[[i-1 for i in self.dlibPoints]]
        features_y = np.array(features_y)[[i-1 for i in self.dlibPoints]]

        # normalize/center the x and y components separately
        features_x = (features_x - np.mean(features_x)) / np.std(features_x)
        features_y = (features_y - np.mean(features_y)) / np.std(features_y)

        # return the feature points in a list [x1, y1, x2, y2, ..., xn, yn]
        return np.reshape(np.concatenate([features_x.reshape(len(features_x),1), features_y.reshape(len(features_x),1)],axis=-1), (len(features_x)*2))
